O objetivo do código neste arquivo é verificar a estacionariedade nas séries

In [3]:
import pandas as pd
import numpy as np
import utils
import sktime.param_est.stationarity as statio

In [4]:
df = pd.read_csv("dados/dados_crus.csv", sep=';', thousands=".", decimal=",")
df.head(10)

,month,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
0,jan/04,0.76,0.68,16.500000,101206857,408357097,850484302,968637026,21851.44,2.8518,...,58072.64,0.4,134.207300,88,124.2,14.696341,184357.2254,291.28,268.94,291.79
1,fev/04,0.61,0.90,16.500000,102857652,411584365,860505531,981337578,21755.02,2.9303,...,58101.64,0.3,255.601393,89,123.4,15.073171,184357.2254,301.39,283.68,316.06
2,mar/04,0.47,0.40,16.387097,101282219,413528931,870826987,989838137,22142.26,2.9055,...,60477.37,0.2,89.185225,84,113.3,16.078049,173974.6570,305.43,286.39,327.01
3,abr/04,0.37,0.21,16.116667,101821601,415095168,876412419,995559726,19607.23,2.9060,...,58565.69,0.2,110.461883,80,108.0,16.454878,173635.2378,304.98,292.74,315.88
4,mai/04,0.51,0.54,16.000000,103074482,429712981,908888294,1012425016,19544.67,3.1004,...,60054.59,0.2,125.354045,97,124.2,15.324390,173873.4968,299.67,277.72,323.91
5,jun/04,0.71,0.56,16.000000,104182650,437868637,916975348,1024499285,21148.91,3.1291,...,65710.98,0.3,136.056788,85,117.1,14.696341,175203.2216,291.16,270.73,300.89
6,jul/04,0.91,0.93,16.000000,104856804,443323859,927486531,1038909105,22336.87,3.0368,...,66770.37,0.5,52.627001,89,118.5,14.068293,175957.5975,290.18,265.42,320.21
7,ago/04,0.69,0.79,16.000000,107417168,451798704,939986824,1047129333,22803.19,3.0029,...,69081.37,0.6,61.805422,84,120.7,14.319512,176585.1903,297.29,277.74,316.01
8,set/04,0.33,0.49,16.125000,111720476,458676647,949435412,1061114243,23245.24,2.8911,...,69115.23,0.7,103.223640,109,128.4,13.691463,175645.7976,294.98,264.04,354.08
9,out/04,0.44,0.32,16.427419,111602434,464622126,959613747,1072347469,23052.18,2.8529,...,69282.96,0.7,137.254963,118,141.9,13.189024,176557.5709,290.02,258.50,348.25


In [5]:
df['month'] = df['month'].map(utils.parse_date)
df['month'] = pd.to_datetime(df['month'], format="%Y-%m-%d")
df.head(10)

,month,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,...,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
0,2004-01-01,0.76,0.68,16.500000,101206857,408357097,850484302,968637026,21851.44,2.8518,...,58072.64,0.4,134.207300,88,124.2,14.696341,184357.2254,291.28,268.94,291.79
1,2004-02-01,0.61,0.90,16.500000,102857652,411584365,860505531,981337578,21755.02,2.9303,...,58101.64,0.3,255.601393,89,123.4,15.073171,184357.2254,301.39,283.68,316.06
2,2004-03-01,0.47,0.40,16.387097,101282219,413528931,870826987,989838137,22142.26,2.9055,...,60477.37,0.2,89.185225,84,113.3,16.078049,173974.6570,305.43,286.39,327.01
3,2004-04-01,0.37,0.21,16.116667,101821601,415095168,876412419,995559726,19607.23,2.9060,...,58565.69,0.2,110.461883,80,108.0,16.454878,173635.2378,304.98,292.74,315.88
4,2004-05-01,0.51,0.54,16.000000,103074482,429712981,908888294,1012425016,19544.67,3.1004,...,60054.59,0.2,125.354045,97,124.2,15.324390,173873.4968,299.67,277.72,323.91
5,2004-06-01,0.71,0.56,16.000000,104182650,437868637,916975348,1024499285,21148.91,3.1291,...,65710.98,0.3,136.056788,85,117.1,14.696341,175203.2216,291.16,270.73,300.89
6,2004-07-01,0.91,0.93,16.000000,104856804,443323859,927486531,1038909105,22336.87,3.0368,...,66770.37,0.5,52.627001,89,118.5,14.068293,175957.5975,290.18,265.42,320.21
7,2004-08-01,0.69,0.79,16.000000,107417168,451798704,939986824,1047129333,22803.19,3.0029,...,69081.37,0.6,61.805422,84,120.7,14.319512,176585.1903,297.29,277.74,316.01
8,2004-09-01,0.33,0.49,16.125000,111720476,458676647,949435412,1061114243,23245.24,2.8911,...,69115.23,0.7,103.223640,109,128.4,13.691463,175645.7976,294.98,264.04,354.08
9,2004-10-01,0.44,0.32,16.427419,111602434,464622126,959613747,1072347469,23052.18,2.8529,...,69282.96,0.7,137.254963,118,141.9,13.189024,176557.5709,290.02,258.50,348.25


In [6]:
# Faz o teste Kwiatkowski-Phillips-Schmidt-Shin (KPSS) para estacionariedade

kpss_results = {}
for col in df.columns:
    kpss_tester = statio.StationarityKPSS()
    X = df[col]
    kpss_results[col] = kpss_tester.fit(X)

C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  res = kpss(
C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of 

In [7]:
for k, v in kpss_results.items():
    print(k)
    print(f"Stationary: {v.stationary_}")
    print(f"P value: {v.pvalue_:.5f}")

month
Stationary: False
P value: 0.01000
ipca
Stationary: True
P value: 0.10000
ipca_15
Stationary: True
P value: 0.10000
selic
Stationary: False
P value: 0.01000
m1
Stationary: False
P value: 0.01000
m2
Stationary: False
P value: 0.01000
m3
Stationary: False
P value: 0.01000
m4
Stationary: False
P value: 0.01000
ibovespa
Stationary: False
P value: 0.01000
cambio
Stationary: False
P value: 0.01000
cambio_real
Stationary: True
P value: 0.05307
ind_varejo
Stationary: False
P value: 0.01000
ind_varejo_ampl
Stationary: False
P value: 0.01000
ind_industria
Stationary: False
P value: 0.01000
ibc_br
Stationary: False
P value: 0.01000
capacidade_industria
Stationary: False
P value: 0.01000
salario_minimo
Stationary: False
P value: 0.01000
energia_total
Stationary: False
P value: 0.01000
energia_residencial
Stationary: False
P value: 0.01000
energia_industrial
Stationary: False
P value: 0.03330
energia_comercial
Stationary: False
P value: 0.01000
energia_outros
Stationary: False
P value: 0.0100

In [8]:
# Identifica séries estacionárias
for k, v in kpss_results.items():
    if v.stationary_:
        print(k)

ipca
ipca_15
cambio_real
oil_usa
oil_eu
bens_capital
prod_caminhoes
el_nino
desemprego


In [9]:
# Faz o teste ADF para estacionariedade
adf_results = {}
for col in df.columns:
    adf_tester = statio.StationarityADF()
    X = df[col]
    adf_results[col] = adf_tester.fit(X)

In [10]:
for k, v in adf_results.items():
    print(k)
    print(f"Stationary: {v.stationary_}")
    print(f"P value: {v.pvalue_}")

month
Stationary: False
P value: 0.979908200895161
ipca
Stationary: True
P value: 8.588722589706786e-12
ipca_15
Stationary: True
P value: 2.8785142091526393e-11
selic
Stationary: False
P value: 0.06699576463321147
m1
Stationary: False
P value: 0.9346172812537812
m2
Stationary: False
P value: 1.0
m3
Stationary: False
P value: 0.9990878242391662
m4
Stationary: False
P value: 0.9989553023350284
ibovespa
Stationary: False
P value: 0.7696188022854988
cambio
Stationary: False
P value: 0.9061908791251828
cambio_real
Stationary: True
P value: 0.005151183336943635
ind_varejo
Stationary: False
P value: 0.9974810142237394
ind_varejo_ampl
Stationary: False
P value: 0.9951795878185864
ind_industria
Stationary: False
P value: 0.46996626451706924
ibc_br
Stationary: False
P value: 0.19818943982085652
capacidade_industria
Stationary: False
P value: 0.28216759922670726
salario_minimo
Stationary: False
P value: 0.9970268608634079
energia_total
Stationary: False
P value: 0.6953268502574788
energia_residen

In [11]:
# Identifica séries estacionárias
for k, v in adf_results.items():
    if v.stationary_:
        print(k)

ipca
ipca_15
cambio_real
oil_usa
oil_eu
el_nino
epu_br
epu_usa


In [12]:
# Testando uma por uma KPSS
new_kpss = statio.StationarityKPSS()
X = df['ipca']
new_kpss.fit(X)
new_kpss.stationary_
new_kpss.pvalue_

C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(


0.1

Conclusão: as seguintes séries são estacionárias:

ipca

ipca_15

cambio_real

oil_usa

oil_eu

bens_capital

prod_caminhoes

el_nino

desemprego
